In [42]:
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime
import pandas as pd

In [43]:
# Create a practice set of random latitude and longitude combinations.
lats = [25.12903645, 25.92017388, 26.62509167, -59.98969384, 37.30571269]
lngs = [-67.59741259, 11.09532135, 74.84233102, -76.89176677, -61.13376282]
lat_lngs = zip(lats, lngs)

In [44]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [45]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=1a01836430fd72a7d3367a55687e8b7b


In [46]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

5

In [49]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | cockburn town
Processing Record 2 of Set 1 | gat
Processing Record 3 of Set 1 | parvatsar
Processing Record 4 of Set 1 | punta arenas
Processing Record 5 of Set 1 | saint george
-----------------------------
Data Retrieval Complete      
-----------------------------


In [50]:
city_data

[{'City': 'Cockburn Town',
  'Lat': 21.4612,
  'Lng': -71.1419,
  'Max Temp': 85.96,
  'Humidity': 70,
  'Cloudiness': 40,
  'Wind Speed': 12.66,
  'Country': 'TC',
  'Date': '2022-08-01 17:36:14'},
 {'City': 'Gat',
  'Lat': 31.61,
  'Lng': 34.7642,
  'Max Temp': 83.39,
  'Humidity': 100,
  'Cloudiness': 18,
  'Wind Speed': 5.46,
  'Country': 'IL',
  'Date': '2022-08-01 17:36:14'},
 {'City': 'Parvatsar',
  'Lat': 26.8833,
  'Lng': 74.7667,
  'Max Temp': 82.49,
  'Humidity': 74,
  'Cloudiness': 78,
  'Wind Speed': 12.5,
  'Country': 'IN',
  'Date': '2022-08-01 17:36:15'},
 {'City': 'Punta Arenas',
  'Lat': -53.15,
  'Lng': -70.9167,
  'Max Temp': 48.31,
  'Humidity': 66,
  'Cloudiness': 75,
  'Wind Speed': 20.71,
  'Country': 'CL',
  'Date': '2022-08-01 17:32:30'},
 {'City': 'Saint George',
  'Lat': 37.1041,
  'Lng': -113.5841,
  'Max Temp': 89.8,
  'Humidity': 47,
  'Cloudiness': 63,
  'Wind Speed': 3,
  'Country': 'US',
  'Date': '2022-08-01 17:33:20'}]

In [51]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cockburn Town,21.4612,-71.1419,85.96,70,40,12.66,TC,2022-08-01 17:36:14
1,Gat,31.6100,34.7642,83.39,100,18,5.46,IL,2022-08-01 17:36:14
2,Parvatsar,26.8833,74.7667,82.49,74,78,12.50,IN,2022-08-01 17:36:15
3,Punta Arenas,-53.1500,-70.9167,48.31,66,75,20.71,CL,2022-08-01 17:32:30
4,Saint George,37.1041,-113.5841,89.80,47,63,3.00,US,2022-08-01 17:33:20


In [54]:
new_column_order = ["City", "Country", "Date","Lat","Lng","Max Temp","Humidity","Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Cockburn Town,TC,2022-08-01 17:36:14,21.4612,-71.1419,85.96,70,40,12.66
1,Gat,IL,2022-08-01 17:36:14,31.6100,34.7642,83.39,100,18,5.46
2,Parvatsar,IN,2022-08-01 17:36:15,26.8833,74.7667,82.49,74,78,12.50
3,Punta Arenas,CL,2022-08-01 17:32:30,-53.1500,-70.9167,48.31,66,75,20.71
4,Saint George,US,2022-08-01 17:33:20,37.1041,-113.5841,89.80,47,63,3.00


In [55]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")